# Índice (tentativo)

- [Introducción](#Introducción)
    - [Objetivos del cuaderno](#Objetivos-del-cuaderno)
    - [Motivación y marco teórico](#Motivación-y-Marco-Teórico)
        - [Dificultad clásica para resolver el problema de búsqueda de período]()
        - [HSP: Problema del subgrupo oculto]()
        - [Búsqueda de Período como instancia del HSP]()
        - [Preguntas exploratorias para la implementación del algoritmo de búsqueda de período]()
- [Diseño de Implementación]()

      


# Introducción

## Objetivos del cuaderno
 - Motivar el desarrollo de la implementación del algoritmo para la solución del problema de búsqueda de período.
 - Entender el problema que resuelve el algoritmo de búsqueda de período cuántico. Cuáles son sus componentes y sus marco teórico.
 - Presentar preguntas exploratorias 

## Motivación y Marco Teórico

La búsqueda del período $r$ de una función periódica $f$ consiste en encontrar el menor entero positivo $r$ tal que:

$$
f(x) = f(x + r) \quad \text{para todo } x \text{ en el dominio.}
$$

Generalmente, consideramos funciones definidas sobre un dominio grande, por ejemplo, un subconjunto de los enteros o números modulares, y con un codominio también amplio.


### ¿Por qué es difícil encontrar el período clásicamente?



Desde un punto de vista clásico, no existe un algoritmo eficiente conocido para encontrar $r$ sin evaluar la función en muchos puntos. Una estrategia natural es:

- Evaluar $f$ en distintos valores $x_1, x_2, \dots, x_n$.
- Buscar colisiones, es decir, pares $(x_i, x_j)$ con $i \neq j$ tales que:
$$f(x_i) = f(x_j)$$

Estas colisiones pueden indicar que la diferencia $|x_i - x_j|$ es un múltiplo del período $r$, lo que ayuda a deducir $r$.

**Número de comparaciones y probabilidad de colisión**

Con $n$ evaluaciones, el número de pares que podemos comparar es:

$$
\binom{n}{2} = \frac{n(n-1)}{2}.
$$

Si suponemos que $f$ toma valores en un conjunto de tamaño $r$ (por ejemplo, $\{0,1,\dots,r-1\}$), la probabilidad de que dos valores evaluados coincidan (colisión) aumenta con $n$.

Este fenómeno es análogo a la *paradoja del cumpleaños*: para que la probabilidad de encontrar al menos una colisión sea significativa (digamos, alrededor del 50%), se necesita aproximadamente:

$$
n \approx 1.2 \sqrt{r}
$$

evaluaciones.

**Complejidad temporal clásica**

En problemas relevantes, como aquellos relacionados con funciones definidas sobre entradas de $n$ bits, el período puede ser tan grande como $2^n$. Esto implica que la búsqueda clásica del período, basada en encontrar colisiones, requiere un número de evaluaciones y comparaciones del orden de:

$$
\mathcal{O}(\sqrt{2^n}) = \mathcal{O}(2^{n/2}),
$$

es decir, tiempo exponencial en $n$.

Esta dificultad es la base para la seguridad de ciertos sistemas criptográficos y motiva la búsqueda de algoritmos más eficientes, como los algoritmos cuánticos.



### Marco Teórico: Problema del Subgrupo Oculto (HSP)

El problema del subgrupo oculto (HSP, por sus siglas en inglés, *Hidden Subgroup Problem*) es un **marco matemático general** que subyace a varios de los algoritmos cuánticos más importantes que demuestran una **ventaja de velocidad exponencial** sobre los mejores algoritmos clásicos conocidos.

#### Fundamentos de Teoría de Grupos para Entender el HSP

Para comprender el **Problema del Subgrupo Oculto (HSP)**, necesitamos familiarizarnos con los siguientes conceptos clave de teoría de grupos:

---
- Grupo $(G, \cdot)$
Un **grupo** es un conjunto $G$ junto con una operación binaria $\cdot$ (como suma o multiplicación), que satisface cuatro propiedades:

1. **Clausura**: $\forall a, b \in G$, se tiene $a \cdot b \in G$.
2. **Asociatividad**: $(a \cdot b) \cdot c = a \cdot (b \cdot c)$.
3. **Elemento neutro**: Existe $e \in G$ tal que $a \cdot e = a$ para todo $a \in G$.
4. **Inversos**: Para cada $a \in G$, existe $a^{-1} \in G$ tal que $a \cdot a^{-1} = e$.

Además, se dice **abeliano** si la operación binaria que define la estructura del grupo es **conmutativa**. En otras palabras, el orden en que se realizan las operaciones con dos elementos no afecta el resultado. 

📌 **Ejemplo**: $(\mathbb{Z}, +)$ es un grupo, donde la operación es la suma. Además es abeliano.

- Subgrupo $K \leq G$
Un **subgrupo** $K$ es un subconjunto de $G$ que forma un grupo bajo la misma operación de $G$.

📌 **Ejemplo**: $2\mathbb{Z} \subseteq \mathbb{Z}$ es un subgrupo de los enteros: suma de pares sigue siendo par, el neutro es $0$, y cada número par tiene inverso aditivo.

- Cosets (Clases laterales)
Dado un subgrupo $K \leq G$ y un elemento $g \in G$, se define la **clase lateral izquierda** como:

$$
gK := \{g \cdot k : k \in K\}
$$

Estas clases particionan el grupo $G$, es decir, **cada elemento de $G$ pertenece a una única clase lateral**.  

📌 **Ejemplo**: En $G = \mathbb{Z}$ y $K = 3\mathbb{Z}$, las clases laterales son:
- $0 + 3\mathbb{Z} = \{\dots, -6, -3, 0, 3, 6, \dots\}$
- $1 + 3\mathbb{Z} = \{\dots, -5, -2, 1, 4, 7, \dots\}$
- $2 + 3\mathbb{Z} = \{\dots, -4, -1, 2, 5, 8, \dots\}$

- Función constante en clases laterales
Una función $f: G \to X$ es **constante en las clases laterales de $K$** si:

$$
f(g_1) = f(g_2) \quad \text{si y sólo si} \quad g_1K = g_2K
$$

📌 Es decir, **$f$ asigna el mismo valor a todos los elementos de una misma clase lateral, y valores distintos entre diferentes clases**.

En el **Problema del Subgrupo Oculto**, **no conocemos el subgrupo $K$**, pero sabemos que **la función $f$ está estructurada de forma que refleja las clases laterales de $K$**.

---

#### Definición del Problema del Subgrupo Oculto (HSP)

El problema del subgrupo oculto se define formalmente como sigue:

> Dada una **función $f$ de un grupo $G$ generado finitamente a un conjunto finito $X$**, con la propiedad de que **$f$ es constante en las cosets de un subgrupo $K$ y distinta en cada coset**. Es decir,  
> 
> $$f(g_1) = f(g_2) \quad \Leftrightarrow \quad g_1 K = g_2 K.$$
> 
> 
> Se tiene acceso a una "caja negra cuántica" que realiza la transformación unitaria  
> 
> $$U_f |g⟩|h⟩ = |g⟩|h \oplus f(g)⟩,$$
> 
> donde $g \in G$, $h \in X$, y $\oplus$ es una operación binaria en $X$.  
> 
> El objetivo es encontrar un conjunto generador del subgrupo oculto $K$.

Ejemplos notables de problemas que son instancias del HSP:
- Factorización de números enteros (algoritmo de Shor)
- Logaritmo discreto
- Búsqueda de órdenes
- Búsqueda de períodos
- Algoritmo de Deutsch-Jozsa
- Problema de Simon

---

#### Ventaja Exponencial para Grupos Abelianos Finitos

Los algoritmos cuánticos que resuelven el HSP para **grupos abelianos finitos** exhiben una clara **ventaja exponencial** en eficiencia computacional gracias a la implementación eficiente de la **Transformada de Fourier Cuántica (QFT)**:

- Para $G$ abeliano y finito, la QFT se puede realizar en **tiempo polinomial en $\log |G|$**.
- Por ejemplo, el algoritmo de Shor para factorización (una instancia del HSP sobre $\mathbb{Z}_N$) tiene complejidad cuántica de  
  $$
  O(n^2 \log n \log \log n),
  $$
  mientras que el mejor algoritmo clásico conocido (el *number field sieve*) tiene complejidad  
  $$
  \exp(\Theta(n^{1/3} \log^{2/3} n)).
  $$
- Esto representa una **aceleración exponencial real**.

---

#### El *Promise Problem*

El HSP es un **problema con promesa** (*promise problem*). Es decir, se **supone garantizado** que la función de entrada $f$ cumple la propiedad de ser constante en las clases laterales de un subgrupo $K$.

- Esta estructura es explotada por el algoritmo cuántico para obtener una solución eficiente.
- Sin esta promesa (es decir, si $f$ es arbitraria), **no se puede garantizar una ventaja cuántica exponencial**.

Esto ilustra que las **ventajas cuánticas no son universales**, sino que aparecen en problemas con estructuras algebraicas aprovechables.

---

#### Principios de Mecánica Cuántica Aprovechados

Los algoritmos para el HSP se basan principalmente en dos principios de la mecánica cuántica:

1. Superposición (*Paralelismo Cuántico*)
- Un estado como  
  $$
  |0⟩^{\otimes n}|0⟩ \xrightarrow{\text{Hadamard}} \frac{1}{\sqrt{2^n}} \sum_{x \in \{0,1\}^n} |x⟩|0⟩
  $$
  permite evaluar $f(x)$ **simultáneamente** para todos los $x$ aplicando la caja negra (oráculo):
  $$
  \frac{1}{\sqrt{2^n}} \sum_{x} |x⟩|f(x)⟩.
  $$

2. Interferencia
- Después de aplicar $U_f$, la información relevante sobre $K$ no está directamente observable.
- Se aplica la **Transformada de Fourier Cuántica (QFT)** sobre el primer registro.
- La QFT permite que ciertas amplitudes se **interfieran constructiva o destructivamente**, revelando la estructura del subgrupo oculto.

---

#### Esquema General del Algoritmo Cuántico para el HSP

1. Preparación del estado inicial
- Dos registros:
  - El primero se pone en superposición uniforme:  
    $$
    \frac{1}{\sqrt{|G|}} \sum_{g \in G} |g⟩
    $$
  - El segundo se inicializa como $|0⟩$ o en una superposición conveniente.

2. Aplicación de la caja negra
- Se aplica $U_f$:
  $$
  \frac{1}{\sqrt{|G|}} \sum_{g \in G} |g⟩|f(g)⟩.
  $$

3. Aplicación de la QFT
- Se aplica la **QFT inversa** al primer registro.
- Esto transforma la información de periodicidad en **picos de amplitud** que codifican información sobre $K$.

4. Medición
- Se mide el primer registro.
- El resultado está relacionado con el **espacio dual** del grupo, ortogonal a $K$.

5. Post-procesamiento clásico
- A partir de múltiples mediciones, se aplica un algoritmo clásico (como el de **fracciones continuas**) para **reconstruir los generadores del subgrupo $K$**.


### Búsqueda de período

El problema de búsqueda de período es un caso especial del HSP. En este contexto, la búsqueda de período se puede ver como la identificación de un subgrupo oculto en un grupo abeliano finito.

El problema se define de la siguiente manera: 

> Suponga que tiene una función $f$ que produce una **salida de un solo bit** y es **periódica**, de tal forma que $f(x + r) = f(x)$ para un período desconocido $r$ (donde $0 < r < N=2^L$).
> Se le proporciona una "caja negra cuántica" (u oráculo) $U_f$ que realiza la transformación unitaria $U_f|x⟩|y⟩ → |x⟩|y \oplus f(x)⟩$ (donde $\oplus$ denota la suma módulo 2). El objetivo es determinar el valor de $r$.

Esta descripción encaja dentro del marco del HSP si se eligen:

- $G= \mathbb{Z}_N$ como grupo abeliano finito,

- $K= \langle r \rangle$ como subgrupo generado por el período $r$,

- $f$ constante en cosets $x+K$, y distinta en cosets diferentes.

Para que esta formulación sea una instancia estricta del HSP sobre $\mathbb{Z}_N$, se requiere que el período $r$ divida a $N$, es decir, $r\mid N$.

Si $r \nmid N$, la función puede seguir siendo periódica en sentido práctico, pero no define una partición del grupo en cosets de un subgrupo válido, y por lo tanto no es formalmente una instancia del HSP sobre $\mathbb{Z}_N$. Sin embargo, el algoritmo cuántico sigue funcionando si se elige $N \gg r$, como sucede en el algoritmo de Shor, porque cuando se aplica la Transformada de Fourier Cuántica (QFT) sobre el grupo $\mathbb{Z}_N$​ a un estado que codifica una función periódica de período $r$, el resultado tiene máxima amplitud (picos) en ciertos estados $m\in \mathbb{Z}_N$​, tales que: $\frac{m}{N} \approx \frac{s}{r}$, donde $s\in Z$ es un entero desconocido entre $0$ y $r−1$. 

### Preguntas Exploratorias 

#### Robustez estructural del algoritmo

1. ¿Qué pasa si el período $r$ es impar o no potencia de 2?

- Relevancia: ¿la estructura de $r$ afecta la fidelidad de la recuperación?
- Implementación: comparar varios valores de $r$: $r=4,5,6,7,8$ todos con mismo $N$.
- Hipótesis: no ser potencia de 2 no impide la recuperación, pero puede generar distribuciones de salida más dispersas.

2. ¿Qué pasa si la función no distingue bien los cosets?

- Relevancia: viola la hipótesis del HSP .
- Implementación: construir una función periódica que repite el mismo valor en algunos cosets distintos.
- Hipótesis: la interferencia cuántica se degrada, disminuye la probabilidad de picos relevantes.

#### Robustez frente a ruido

3. ¿Qué ocurre si se corrompe la función $f$ en un subconjunto del dominio?

- Relevancia: simula errores en el oráculo.
- Implementación: alterar aleatoriamente algunos valores de $f$.
- Hipótesis: el algoritmo tolera corrupción leve (estudiar porcentaje), pero errores en múltiples cosets destruyen la señal.

#### Variantes
4. ¿Qué ocurre si la función tiene más de un período (función multiperiódica)?

- Relevancia: generalización interesante.
- Implementación: definir una función como $f(x)=g(x mod r_1)+h(x mod r_2)$.
- Hipótesis: la QFT muestra varios picos y se puede identificar un mínimo común múltiplo, aunque el algoritmo estándar no lo capture directamente.

5. ¿Qué hace el algoritmo con una función $f(x)=x+g(x)$ con g una función periódica?

- Relevancia: Esta función no es periódica, por lo tanto viola la promesa del HSP. Sirve como contraejemplo para evaluar qué pasa cuando la función no tiene simetría de cosets.
- Implementación: Definir $g(x)=(−1)^{x mod r}$ u otra periódica. Aplicar el algoritmo de búsqueda de período con esta $f(x)$.
- Hipótesis: La QFT no debería producir picos definidos. Se espera una distribución difusa o aleatoria y fracaso del algoritmo al intentar recuperar un período.

- ¿El algoritmo encuentra algún falso positivo?
- ¿Qué distribución produce la QFT?
- ¿Cómo varía al cambiar el período de $g$ o al eliminar la parte lineal $x$?

6. ¿Qué hace el algoritmo con una función $f(x)=x \mod r +g(x)$ con g una función periódica?
- Relevancia: Esta función sí puede ser periódica si $g$ tiene el mismo período $r$. Es un caso no trivial que puede seguir cumpliendo con la estructura del HSP.
- Implementación: Tomar $g(x)$ de período $r$ o $r′$, comparar ambos casos. Aplicar el algoritmo cuántico y observar la distribución tras la QFT.
- Hipótesis:
    - Si $per⁡(g)=r$, entonces $f$ es periódica $\implies$ se espera recuperación correcta del período.
    - Si $per⁡(g)\neq r$, la periodicidad se altera $\implies$ la QFT podría mostrar picos espurios o desfasados.